In [2]:
from tqdm import tqdm
import tensorflow as tf
import json
import tensorflow_text as text
import os
import numpy as np
import pandas as pd
import pickle

In [3]:
root='C:/Users/delser/'
#name='all-HRMS-loss-header-train-refine_prepro.tsv'
name='all_HRMS_train_24012023_cddd_refine_s_ord.tsv'
#name='all_HRMS_valid_24012023_cddd_refine_s.tsv'
os.path.join(root, name)
df_train = pd.read_csv(os.path.join(root, name), sep="\t") 

df_train

,Unnamed: 0,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,59067,NIST16674,[M-H2O+H]+,59.0491,"[31.02, 39.02, 41.04, 42.01, 42.03, 43.02, 43....","[0.4096996996996997, 0.542002002002002, 1.0, 0...","[15.019100000000002, 15.0291, 16.0091000000000...","[0.747997997997998, 0.3757957957957958, 0.2046...",DNIAPMSPPWPWGF-UHFFFAOYSA-N,CC(O)CO,...,0,0,0,0,0,0,0,0,0,0
1,57392,NIST10860,[M+H]+,60.0556,"[42.02, 43.03, 43.05, 44.04, 60.06, 91.05]","[0.015395395395395396, 1.0, 0.0439039039039039...","[16.0156, 17.0056, 17.025599999999997, 18.0355...","[0.016196196196196194, 0.043903903903903904, 1...",ZRALSGWEFCBTJO-UHFFFAOYSA-N,N=C(N)N,...,0,0,0,0,0,0,0,0,0,0
2,60462,NIST2337,[M+H]+,62.0600,"[29.0, 41.03, 42.01, 42.03, 43.02, 43.03, 43.0...","[0.06719719719719719, 0.013403403403403404, 0....","[17.03, 18.010000000000005, 18.050000000000004...","[0.9176976976976977, 1.0, 0.0492992992992993, ...",HZAXFHJVJLSVMW-UHFFFAOYSA-N,NCCO,...,0,0,0,0,0,0,0,0,0,0
3,59346,NIST17827,[M+H]+,69.0447,"[40.02, 41.02, 41.03, 42.03, 52.02, 67.03, 69.04]","[0.12420420420420421, 0.5763963963963964, 0.22...","[17.024700000000003, 27.014700000000005, 28.01...","[0.016996996996996996, 1.0, 0.2213013013013013...",RAXXELZNTBOGNW-UHFFFAOYSA-N,c1c[nH]cn1,...,0,0,0,0,0,0,0,0,0,0
4,59445,NIST18493,[M-H2O+H]+,69.0699,"[37.01, 38.01, 39.02, 41.04, 50.01, 51.02, 53....","[0.21779779779779782, 0.3234034034034034, 1.0,...","[16.029900000000005, 18.0499, 19.0599000000000...","[0.0575975975975976, 0.0556956956956957, 0.187...",BTSIZIIPFNVMHF-ARJAWSKDSA-N,CCC=CCO,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83353,22573,CCMSLIB00005721981,[2M+Na]+,1169.7900,"[56.182598, 69.005898, 78.622704, 144.092102, ...","[0.19090909090909092, 0.21818181818181817, 0.2...","[127.77193399999987, 262.97749999999996, 339.0...","[0.2818181818181818, 0.2909090909090909, 0.236...",AKMKWHBDPUOPSK-UHFFFAOYSA-N,CCCCCCCCCCC(C)C(O)C(C)C(=O)NC(C(=O)NC(C(=O)NC(...,...,0,0,0,0,0,0,0,0,0,6
83354,22839,CCMSLIB00005722538,[2M+Na]+,1177.6800,"[55.7155, 57.708801, 59.571999, 59.917801, 225...","[0.029411764705882353, 0.041176470588235294, 0...",[334.0114820000001],[0.04264705882352941],XIYSEKITPHTMJT-UHFFFAOYSA-N,C=CCC1OC(=O)CCNC(=O)C(C)N(C)C(=O)C(C(C)C)N(C)C...,...,0,0,0,0,0,0,0,0,0,0
83355,22936,CCMSLIB00005722698,[2M+Na]+,1179.4200,"[55.522701, 66.6054, 69.357002, 89.308403, 110...","[0.03230769230769231, 0.03230769230769231, 0.0...",[125.31392100000016],[0.05384615384615385],AVAQRWVGWWNBIN-CUAUGLLKSA-N,COc1cc(OC)c2c(O)c3c(c(-c4c5c(c(O)c6c(OC)cc(OC)...,...,2,0,0,0,0,0,0,0,0,0
83356,23484,CCMSLIB00005724769,[2M+H]+,1179.5500,"[168.047195, 171.11702, 186.058151, 187.061752...","[0.0508194359063266, 0.013158516161691445, 1.0...","[55.122020999999904, 124.42585399999984]","[0.01936193920303824, 0.020131051131350296]",YLDBWOUPHZGCJG-WKZPBKTESA-N,CC1CC2(NCCS2)C2(O)OC3CC4(C=O)C(CCC5C4CCC4(C)C(...,...,0,0,0,0,0,0,0,1,0,0


In [3]:
x=df_train["intensities"].to_list()
max([len(json.loads(i)) for i in x])

250

In [4]:
df_train=df_train.dropna()
df_train

,Unnamed: 0,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,59067,NIST16674,[M-H2O+H]+,59.0491,"[31.02, 39.02, 41.04, 42.01, 42.03, 43.02, 43....","[0.4096996996996997, 0.542002002002002, 1.0, 0...","[15.019100000000002, 15.0291, 16.0091000000000...","[0.747997997997998, 0.3757957957957958, 0.2046...",DNIAPMSPPWPWGF-UHFFFAOYSA-N,CC(O)CO,...,0,0,0,0,0,0,0,0,0,0
1,57392,NIST10860,[M+H]+,60.0556,"[42.02, 43.03, 43.05, 44.04, 60.06, 91.05]","[0.015395395395395396, 1.0, 0.0439039039039039...","[16.0156, 17.0056, 17.025599999999997, 18.0355...","[0.016196196196196194, 0.043903903903903904, 1...",ZRALSGWEFCBTJO-UHFFFAOYSA-N,N=C(N)N,...,0,0,0,0,0,0,0,0,0,0
2,60462,NIST2337,[M+H]+,62.0600,"[29.0, 41.03, 42.01, 42.03, 43.02, 43.03, 43.0...","[0.06719719719719719, 0.013403403403403404, 0....","[17.03, 18.010000000000005, 18.050000000000004...","[0.9176976976976977, 1.0, 0.0492992992992993, ...",HZAXFHJVJLSVMW-UHFFFAOYSA-N,NCCO,...,0,0,0,0,0,0,0,0,0,0
3,59346,NIST17827,[M+H]+,69.0447,"[40.02, 41.02, 41.03, 42.03, 52.02, 67.03, 69.04]","[0.12420420420420421, 0.5763963963963964, 0.22...","[17.024700000000003, 27.014700000000005, 28.01...","[0.016996996996996996, 1.0, 0.2213013013013013...",RAXXELZNTBOGNW-UHFFFAOYSA-N,c1c[nH]cn1,...,0,0,0,0,0,0,0,0,0,0
4,59445,NIST18493,[M-H2O+H]+,69.0699,"[37.01, 38.01, 39.02, 41.04, 50.01, 51.02, 53....","[0.21779779779779782, 0.3234034034034034, 1.0,...","[16.029900000000005, 18.0499, 19.0599000000000...","[0.0575975975975976, 0.0556956956956957, 0.187...",BTSIZIIPFNVMHF-ARJAWSKDSA-N,CCC=CCO,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83353,22573,CCMSLIB00005721981,[2M+Na]+,1169.7900,"[56.182598, 69.005898, 78.622704, 144.092102, ...","[0.19090909090909092, 0.21818181818181817, 0.2...","[127.77193399999987, 262.97749999999996, 339.0...","[0.2818181818181818, 0.2909090909090909, 0.236...",AKMKWHBDPUOPSK-UHFFFAOYSA-N,CCCCCCCCCCC(C)C(O)C(C)C(=O)NC(C(=O)NC(C(=O)NC(...,...,0,0,0,0,0,0,0,0,0,6
83354,22839,CCMSLIB00005722538,[2M+Na]+,1177.6800,"[55.7155, 57.708801, 59.571999, 59.917801, 225...","[0.029411764705882353, 0.041176470588235294, 0...",[334.0114820000001],[0.04264705882352941],XIYSEKITPHTMJT-UHFFFAOYSA-N,C=CCC1OC(=O)CCNC(=O)C(C)N(C)C(=O)C(C(C)C)N(C)C...,...,0,0,0,0,0,0,0,0,0,0
83355,22936,CCMSLIB00005722698,[2M+Na]+,1179.4200,"[55.522701, 66.6054, 69.357002, 89.308403, 110...","[0.03230769230769231, 0.03230769230769231, 0.0...",[125.31392100000016],[0.05384615384615385],AVAQRWVGWWNBIN-CUAUGLLKSA-N,COc1cc(OC)c2c(O)c3c(c(-c4c5c(c(O)c6c(OC)cc(OC)...,...,2,0,0,0,0,0,0,0,0,0
83356,23484,CCMSLIB00005724769,[2M+H]+,1179.5500,"[168.047195, 171.11702, 186.058151, 187.061752...","[0.0508194359063266, 0.013158516161691445, 1.0...","[55.122020999999904, 124.42585399999984]","[0.01936193920303824, 0.020131051131350296]",YLDBWOUPHZGCJG-WKZPBKTESA-N,CC1CC2(NCCS2)C2(O)OC3CC4(C=O)C(CCC5C4CCC4(C)C(...,...,0,0,0,0,0,0,0,1,0,0


In [5]:
df_train=df_train.loc[df_train.loss_mzs.apply(str) != "[]"]
df_train

,Unnamed: 0,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,59067,NIST16674,[M-H2O+H]+,59.0491,"[31.02, 39.02, 41.04, 42.01, 42.03, 43.02, 43....","[0.4096996996996997, 0.542002002002002, 1.0, 0...","[15.019100000000002, 15.0291, 16.0091000000000...","[0.747997997997998, 0.3757957957957958, 0.2046...",DNIAPMSPPWPWGF-UHFFFAOYSA-N,CC(O)CO,...,0,0,0,0,0,0,0,0,0,0
1,57392,NIST10860,[M+H]+,60.0556,"[42.02, 43.03, 43.05, 44.04, 60.06, 91.05]","[0.015395395395395396, 1.0, 0.0439039039039039...","[16.0156, 17.0056, 17.025599999999997, 18.0355...","[0.016196196196196194, 0.043903903903903904, 1...",ZRALSGWEFCBTJO-UHFFFAOYSA-N,N=C(N)N,...,0,0,0,0,0,0,0,0,0,0
2,60462,NIST2337,[M+H]+,62.0600,"[29.0, 41.03, 42.01, 42.03, 43.02, 43.03, 43.0...","[0.06719719719719719, 0.013403403403403404, 0....","[17.03, 18.010000000000005, 18.050000000000004...","[0.9176976976976977, 1.0, 0.0492992992992993, ...",HZAXFHJVJLSVMW-UHFFFAOYSA-N,NCCO,...,0,0,0,0,0,0,0,0,0,0
3,59346,NIST17827,[M+H]+,69.0447,"[40.02, 41.02, 41.03, 42.03, 52.02, 67.03, 69.04]","[0.12420420420420421, 0.5763963963963964, 0.22...","[17.024700000000003, 27.014700000000005, 28.01...","[0.016996996996996996, 1.0, 0.2213013013013013...",RAXXELZNTBOGNW-UHFFFAOYSA-N,c1c[nH]cn1,...,0,0,0,0,0,0,0,0,0,0
4,59445,NIST18493,[M-H2O+H]+,69.0699,"[37.01, 38.01, 39.02, 41.04, 50.01, 51.02, 53....","[0.21779779779779782, 0.3234034034034034, 1.0,...","[16.029900000000005, 18.0499, 19.0599000000000...","[0.0575975975975976, 0.0556956956956957, 0.187...",BTSIZIIPFNVMHF-ARJAWSKDSA-N,CCC=CCO,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83353,22573,CCMSLIB00005721981,[2M+Na]+,1169.7900,"[56.182598, 69.005898, 78.622704, 144.092102, ...","[0.19090909090909092, 0.21818181818181817, 0.2...","[127.77193399999987, 262.97749999999996, 339.0...","[0.2818181818181818, 0.2909090909090909, 0.236...",AKMKWHBDPUOPSK-UHFFFAOYSA-N,CCCCCCCCCCC(C)C(O)C(C)C(=O)NC(C(=O)NC(C(=O)NC(...,...,0,0,0,0,0,0,0,0,0,6
83354,22839,CCMSLIB00005722538,[2M+Na]+,1177.6800,"[55.7155, 57.708801, 59.571999, 59.917801, 225...","[0.029411764705882353, 0.041176470588235294, 0...",[334.0114820000001],[0.04264705882352941],XIYSEKITPHTMJT-UHFFFAOYSA-N,C=CCC1OC(=O)CCNC(=O)C(C)N(C)C(=O)C(C(C)C)N(C)C...,...,0,0,0,0,0,0,0,0,0,0
83355,22936,CCMSLIB00005722698,[2M+Na]+,1179.4200,"[55.522701, 66.6054, 69.357002, 89.308403, 110...","[0.03230769230769231, 0.03230769230769231, 0.0...",[125.31392100000016],[0.05384615384615385],AVAQRWVGWWNBIN-CUAUGLLKSA-N,COc1cc(OC)c2c(O)c3c(c(-c4c5c(c(O)c6c(OC)cc(OC)...,...,2,0,0,0,0,0,0,0,0,0
83356,23484,CCMSLIB00005724769,[2M+H]+,1179.5500,"[168.047195, 171.11702, 186.058151, 187.061752...","[0.0508194359063266, 0.013158516161691445, 1.0...","[55.122020999999904, 124.42585399999984]","[0.01936193920303824, 0.020131051131350296]",YLDBWOUPHZGCJG-WKZPBKTESA-N,CC1CC2(NCCS2)C2(O)OC3CC4(C=O)C(CCC5C4CCC4(C)C(...,...,0,0,0,0,0,0,0,1,0,0


In [6]:
def positional_encoding(max_position, d_model, min_freq=1e-6):
    position = np.arange(max_position)
    freqs = min_freq**(2*(np.arange(d_model)//2)/d_model)
    pos_enc = position.reshape(-1,1)*freqs.reshape(1,-1)
    pos_enc[:, ::2] = np.cos(pos_enc[:, ::2])
    pos_enc[:, 1::2] = np.sin(pos_enc[:, 1::2])
    return pos_enc

def trun_n_d(n,d):
    return (  n if not n.find('.')+1 else n[:n.find('.')+d+1]  )

In [7]:
P=positional_encoding(200000,256, min_freq=1e2)
#np.save('positions_512_1e2.npy',P)

In [8]:
def prepro_specs_train(df):
    valid=[]
    precs=df['precursor_mz'].to_list()
    mzs=df['mzs'].to_list()
    ints=df['intensities'].to_list()
    loss_mzs=df['loss_mzs'].to_list()
    loss_ints=df['loss_intensities'].to_list()
    for one_pre,one_mzs,one_ints,one_loss,one_loss_ints in tqdm(zip(precs,mzs,ints,loss_mzs,loss_ints)):
        mz_list=[round(float(trun_n_d(str(one_pre),2))*100)] # add precursor mz
        intes_list=[2.0] # add precursor int
        res = dict(zip(json.loads(one_mzs)+json.loads(one_loss), json.loads(one_ints)+json.loads(one_loss_ints)))  # order by mzs
        res=dict(sorted(res.items()))
        for m,i in zip(list(res.keys()), list(res.values())): # change this from mgf from matchms
            mz=round(float(trun_n_d(str(m),2))*100)
            mz_list.append(mz)
            intens=round(i,4)
            intes_list.append(intens)
        int_mzs=[intes_list,mz_list]   
        valid.append(int_mzs) # put intesities at first
    return tf.ragged.constant(valid)

In [9]:
%%time
train=prepro_specs_train(df_train)

83358it [01:07, 1240.89it/s]


CPU times: total: 3min 32s
Wall time: 3min 32s


In [10]:
dimn=256
def encoding(rag_tensor,P,dimn):
    to_pad=[]
    for sample in rag_tensor:
        all_dim=[sample[0].numpy().tolist()]
        pos_enc=[P[int(i)-1] for i in sample[1].numpy().tolist()]
        for dim in range(dimn):
            dim_n=[i[dim] for i in pos_enc]
            all_dim.append(dim_n)
        to_pad.append(all_dim)
    to_pad=[tf.keras.preprocessing.sequence.pad_sequences(i,maxlen=501,dtype='float32',padding='post',truncating='post',value=10) for i in to_pad]
    to_pad=np.stack((to_pad))
    to_pad=np.swapaxes(to_pad, 1, -1)
    return to_pad

In [11]:
%%time
xtrain=encoding(train,P,dimn)

CPU times: total: 1h 35min
Wall time: 1h 18min 19s


In [12]:
%%time
np.save('tcn_train_seq_sin256_2401_ord.npy',xtrain)

CPU times: total: 18min 16s
Wall time: 18min 16s
